# Projeto YOLOv8 - Versão para Google Colab (CPU-only)

Este notebook permite treinar e testar um modelo de detecção de objetos YOLOv8 diretamente no Google Colab, configurado para rodar exclusivamente na CPU.

## 1. Configuração do Ambiente

Primeiro, vamos instalar as bibliotecas necessárias e baixar o modelo pré-treinado. Este notebook está configurado para usar a CPU.

In [1]:
%pip install ultralytics pycocotools torch torchvision wget
import wget
import torch

print("Baixando o modelo yolov8n.pt...")
wget.download('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt')

device = torch.device('cpu')
print('Running on CPU')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=12450f8f8fd8d669be39bb3f0c7e6d711ff3eb9334a7c030f130a27a6dde2eae
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget
Baixando o modelo yolov8n.pt...
Running on CPU


## 2. Treinamento do Modelo

Agora, vamos treinar o modelo YOLOv8 com as classes `car` e `motorcycle` do dataset COCO128. O dataset será baixado automaticamente. O treinamento será executado na CPU.

In [3]:
from ultralytics import YOLO

def train_model(device):
    model = YOLO('yolov8n.pt')

    print(f"\nIniciando o treinamento em {device.type.upper()}...")
    print("Isso pode levar alguns minutos, mesmo com o dataset de teste.")
    print(f"Parâmetros: epochs=3, imgsz=320")

    # Treina o modelo usando o dataset COCO128, focando nas classes 2 (car) e 3 (motorcycle)
    results = model.train(data='coco128.yaml', epochs=3, imgsz=320, classes=[2, 3], device='cpu')

    print("---------------------------------")
    print("✅ Treinamento concluído com sucesso!")
    print(f"📈 Resultados e pesos do modelo foram salvos em: {results.save_dir}")
    print("---------------------------------")
    return results.save_dir

model_save_dir = train_model(device)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

Iniciando o treinamento em CPU...
Isso pode levar alguns minutos, mesmo com o dataset de teste.
Parâmetros: epochs=3, imgsz=320
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=[2, 3], close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscrip

## 3. Detecção de Objetos

Use o modelo que acabamos de treinar para detectar objetos em uma nova imagem.

In [4]:
import os
from ultralytics import YOLO
from PIL import Image
from google.colab import files
from IPython.display import display

def detect_objects(model_path, image_path):
    try:
        model = YOLO(model_path)
        results = model(image_path)

        for r in results:
            im_array = r.plot()
            im = Image.fromarray(im_array[..., ::-1])

            print("Exibindo imagem com detecções:")
            display(im)

            result_filename = 'result.jpg'
            im.save(result_filename)
            print(f"Imagem com detecções salva como '{result_filename}'")

    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado em '{image_path}' ou '{model_path}'")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# Caminho para o modelo treinado (geralmente o melhor está em 'weights/best.pt')
trained_model_path = os.path.join(model_save_dir, 'weights/best.pt')

if not os.path.exists(trained_model_path):
    print(f'O modelo treinado não foi encontrado em {trained_model_path}')
    print('Por favor, certifique-se de que o treinamento foi concluído com sucesso.')
else:
    # Upload da imagem para detecção
    print("Faça o upload da imagem que deseja analisar.")
    uploaded_image = files.upload()

    if uploaded_image:
        image_filename = next(iter(uploaded_image))
        print(f"Analisando a imagem: {image_filename}")
        detect_objects(model_path=trained_model_path, image_path=image_filename)

Faça o upload da imagem que deseja analisar.
